In [30]:
import os 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline 
import warnings 
warnings.filterwarnings('ignore')
#Modeling 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR 
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from catboost import CatBoostRegressor 
from xgboost import XGBRegressor 
import warnings 



# Model Training 

In [6]:
df = pd.read_csv('data/cleaned_data.csv')

In [23]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score,average
0,female,group_B,bachelors_degree,standard,none,72,72,74,218,72.67
1,female,group_C,some_college,standard,completed,69,90,88,247,82.33
2,female,group_B,masters_degree,standard,none,90,95,93,278,92.67
3,male,group_A,associates_degree,free_reduced,none,47,57,44,148,49.33
4,male,group_C,some_college,standard,none,76,78,75,229,76.33


In [24]:
print("Categories in 'gender' variable:     ",end=" " )
print(df['gender'].unique())

print("Categories in 'race_ethnicity' variable:  ",end=" ")
print(df['race_ethnicity'].unique())

print("Categories in'parental level of education' variable:",end=" " )
print(df['parental_level_of_education'].unique())

print("Categories in 'lunch' variable:     ",end=" " )
print(df['lunch'].unique())

print("Categories in 'test preparation course' variable:     ",end=" " )
print(df['test_preparation_course'].unique())

Categories in 'gender' variable:      ['female' 'male']
Categories in 'race_ethnicity' variable:   ['group_B' 'group_C' 'group_A' 'group_D' 'group_E']
Categories in'parental level of education' variable: ['bachelors_degree' 'some_college' 'masters_degree' 'associates_degree'
 'high_school' 'some_high_school']
Categories in 'lunch' variable:      ['standard' 'free_reduced']
Categories in 'test preparation course' variable:      ['none' 'completed']


# Separate Features and Target

In [8]:
X = df.drop(columns=['average'], axis= 1)

In [9]:
y = df['average']

In [10]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score
0,female,group_B,bachelors_degree,standard,none,72,72,74,218
1,female,group_C,some_college,standard,completed,69,90,88,247
2,female,group_B,masters_degree,standard,none,90,95,93,278
3,male,group_A,associates_degree,free_reduced,none,47,57,44,148
4,male,group_C,some_college,standard,none,76,78,75,229


In [11]:
y.head()

0    72.67
1    82.33
2    92.67
3    49.33
4    76.33
Name: average, dtype: float64

# Create Column Transformer with 3 types of Transformers 

In [14]:
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler() 
oh_transformer = OneHotEncoder() 

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features), 
        ("StandardScaler", numeric_transformer, num_features),
        
    ]
           
)

In [15]:
X = preprocessor.fit_transform(X)

## Train and Test Split 

In [21]:
X_train, X_test, y_train, y_test  = train_test_split(X,y, test_size=0.2, random_state = 37)
X_train.shape, X_test.shape

((800, 21), (200, 21))

# Model Performance Metrics 

In [34]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted),
    mse = mean_squared_error(true, predicted), 
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square 

# Models 

In [40]:
import pandas as pd

models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

# Create an empty DataFrame
results_df = pd.DataFrame(columns=["Model", "Train RMSE", "Test RMSE", "Train MAE", "Test MAE", "Train R2", "Test R2"])

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    # Append results to DataFrame
    temp_df = pd.DataFrame({
        "Model": [list(models.keys())[i]],
        "Train RMSE": [model_train_rmse],
        "Test RMSE": [model_test_rmse],
        "Train MAE": [model_train_mae],
        "Test MAE": [model_test_mae],
        "Train R2": [model_train_r2],
        "Test R2": [model_test_r2]
    })
    
    results_df = pd.concat([results_df, temp_df], ignore_index=True)

# Calculate the absolute difference between train and test RMSE
results_df['RMSE Difference'] = abs(results_df['Train RMSE'] - results_df['Test RMSE'])

# Sort the DataFrame based on the RMSE Difference
results_df = results_df.sort_values(by='RMSE Difference', ascending=True)

# Display the sorted DataFrame
print(results_df)



                     Model    Train RMSE  Test RMSE                 Train MAE   
2                    Ridge  6.390610e-03   0.006490   (0.005149511904506374,)  \
0        Linear Regression  2.658223e-03   0.002841  (0.0022329740524303075,)   
1                    Lasso  1.014759e+00   1.087984      (0.815110600525376,)   
8       AdaBoost Regressor  1.114046e+00   1.529091     (0.8823370617443672,)   
3    K-Neighbors Regressor  2.082902e+00   2.668773     (1.6462825000000003,)   
6             XGBRegressor  1.553354e-02   0.734358    (0.01100969886779783,)   
4            Decision Tree  2.161031e-15   0.772400  (6.572520305780927e-16,)   
5  Random Forest Regressor  1.085891e-01   0.925913   (0.023241625000004252,)   
7    CatBoosting Regressor  1.156240e-01   1.026135    (0.09284508610221678,)   

                   Test MAE  Train R2   Test R2  RMSE Difference  
2    (0.00516039364249675,)  1.000000  1.000000         0.000100  
0  (0.0024502792358410373,)  1.000000  1.000000        